In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Reasoning Engine in Vertex AI - Chat history management
@forusone (shins777@gmail.com)

This colab explains how to deal with chat history being stored in firestore.   

Reference : https://python.langchain.com/v0.2/docs/integrations/memory/google_firestore/


## Init Vertex AI SDK

In [1]:
!pip install --upgrade --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    "google-cloud-firestore" \
    "langchain-google-firestore" \
    cloudpickle==3.0.0 \
    pydantic==2.7.4 \
    requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.0/409.0 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.3/94.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00


In [11]:
import sys
from IPython.display import display, Markdown

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Configuration and initialization

In [3]:
# Static configuration
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://reasoning_7424"
MODEL = "gemini-1.5-flash"

# Firestore
DATABASE="chat-history"
COLLECTION="history"


### Initialize Vertex AI

In [4]:
import vertexai
from vertexai.preview import reasoning_engines

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

## Firestore setting for chat history

Before executing the following cell, you have to set up firestore database in GCP Console.
* Create database : https://cloud.google.com/firestore/docs/manage-databases

### Helper method to get session history

In [34]:
# refer to : https://python.langchain.com/v0.2/docs/integrations/memory/google_firestore/#custom-client

def get_session_history(session_id: str):
    """
    Retrieve chat history data with session id for each conversation from Firestore on GCP
    """

    from langchain_google_firestore import FirestoreChatMessageHistory
    from google.cloud import firestore

    client = firestore.Client(project=PROJECT_ID, database=DATABASE)

    chat_history = FirestoreChatMessageHistory(
                      client=client,
                      session_id=session_id,
                      collection=COLLECTION,
                      encode_message=False,
    )

    return chat_history

def get_converstion(chat_history):
  """
  Get conversation from chat history
  """

  from langchain.schema import HumanMessage, AIMessage
  import langchain_core.messages

  for message in chat_history.messages:
      if type(message) is HumanMessage:
        print(f"Human : {message.content}")
      elif type(message) is AIMessage:
        print(f"AI : {message.content}")


def interactive_chat(query, session_id):
  """
  Chat with agent with chat history.
  """
  respone = agent.query(input=query,
                        config={"configurable": {"session_id": session_id}},
  )
  return respone['output']


### Unit test of Firestore

In [28]:
session_id = "conv-003"

chat_history = get_session_history(session_id)
chat_history.clear()
print(chat_history.messages)

[]


## Integrate reasoning engine and chat history

### Tools definition

In [8]:
# Define Google Search tool
from vertexai.generative_models import grounding, Tool
grounded_search_tool = Tool.from_google_search_retrieval(
    grounding.GoogleSearchRetrieval()
)

### Agent - Local unit test

In [17]:
from vertexai.preview.generative_models import ToolConfig

# Agent Define
agent = reasoning_engines.LangchainAgent(
    model=MODEL,
    tools=[grounded_search_tool,],
    chat_history=get_session_history,  # function to get FirestoreChatMessageHistory
    agent_executor_kwargs={"return_intermediate_steps": False},
)

### Add chat come from interaction into chat history


In [25]:
# def interactive_chat(query, session_id):
#   """
#   Chat with agent with chat history.
#   """
#   respone = agent.query(input=query,
#                         config={"configurable": {"session_id": session_id}},
#   )
#   return respone['output']


In [29]:
from time import perf_counter

while True:
  query = input('사용자: ')

  if query == '종료': break

  t1_start = perf_counter()
  output = interactive_chat(query, session_id)
  t1_end = perf_counter()

  display(Markdown(f"{t1_end-t1_start}"))
  display(Markdown(f"AI Agent : {output}"))
  print(f"------------------------------------ ")

chat_history = get_session_history(session_id)
chat_history.clear()
print(chat_history.messages)

사용자: 안녕하세요


2.3419530820001455

AI Agent : "안녕하세요"는 한국어에서 가장 일반적인 인사말입니다.  "안녕하세요"는 "무탈하시오?"와 같이 "잘 지내시나요?"라고 해석할 수 있습니다.  "안녕하세요"는 "안녕하다"에서 온 말인데, "안녕하다"는 "평화롭다" 또는 "편안하다"는 뜻입니다.  그래서 "안녕하세요"는 "잘 지내시나요?" 또는 "평안하신가요?"와 같은 의미를 지닙니다.


------------------------------------ 
사용자: 현재 한국의 경제 상황에 대해서 설명해주세요.


4.776272721999931

...

AI Agent : 한국 내수 부진의 주된 원인은 다음과 같습니다:

* **고금리:** 높은 금리로 인해 기업 투자는 위축되고 가계 소비는 줄어들고 있습니다. 높은 금리는 기업의 투자 비용을 증가시키고, 가계의 부채 상환 부담을 높여 소비를 위축시킵니다. 
* **가계 부채:** 높은 가계 부채는 가계 소비를 위축시키는 주요 원인입니다. 높은 부채 부담은 소비 지출을 줄이고, 가계의 불안감을 증가시킵니다.
* **경제 정책 불확실성:** 정치적 불안정, 경제 정책 변화 등으로 인한 경제 정책 불확실성은 기업 투자와 소비 심리를 위축시킵니다. 
* **소득 불평등 심화:** 소득 불평등 심화는 내수 부진의 주요 원인 중 하나입니다. 소득 불평등이 심화되면 저소득층의 소비 지출이 줄어들고, 경제 성장 잠재력이 감소합니다.
* **수출 의존도 심화:** 한국 경제는 수출에 대한 의존도가 높습니다. 수출 부진은 내수 부진으로 이어질 수 있습니다. 
* **인구 감소:** 생산 가능 인구 감소는 내수 시장 규모를 축소시키는 요인입니다. 

내수 부진을 해결하기 위해서는 고금리 해소, 가계 부채 관리, 경제 정책 안정, 소득 불평등 완화, 내수 산업 육성 등 다각적인 노력이 필요합니다. 


------------------------------------ 
사용자: 지금 어느나라 이야기를 하고 있나요 ?


1.7679952910000338

AI Agent :  우리는 한국 경제 상황에 대해 이야기하고 있습니다. 


------------------------------------ 
사용자: 종료


In [36]:
session_id = "conv-003"
chat_history = get_session_history(session_id)
chat_history.messages

[HumanMessage(content='안녕하세요', additional_kwargs={}, response_metadata={}),
 AIMessage(content='"안녕하세요"는 한국어에서 가장 일반적인 인사말입니다.  "안녕하세요"는 "무탈하시오?"와 같이 "잘 지내시나요?"라고 해석할 수 있습니다.  "안녕하세요"는 "안녕하다"에서 온 말인데, "안녕하다"는 "평화롭다" 또는 "편안하다"는 뜻입니다.  그래서 "안녕하세요"는 "잘 지내시나요?" 또는 "평안하신가요?"와 같은 의미를 지닙니다.\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='현재 한국의 경제 상황에 대해서 설명해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content=' 한국의 경제 상황은 현재 긍정적이지만, 일부 어려움을 겪고 있습니다. \n\n* **긍정적인 측면:**\n    * **수출:** 한국의 수출은 ICT 제품을 중심으로 양호한 흐름을 유지하고 있습니다. 특히 반도체 수출이 높은 증가세를 기록하고 있습니다. \n    * **경제 성장:** 2024년 한국 경제 성장률은 2.2%에서 2.7%로 상향 조정되었습니다. 이는 1분기 경제 성장률이 예상보다 높았고, 수출 경기 회복이 내수 부진을 일부 완화했기 때문입니다.\n    * **설비 투자:** 금리 인하와 반도체 경기 호조세로 설비 투자가 증가할 것으로 예상됩니다. 특히 반도체 관련 투자가 확대되고 있습니다.\n    * **경상수지:** 수출 증가세 둔화와 내수 회복에도 불구하고 교역 조건 개선으로 경상수지 흑자 폭이 확대될 전망입니다. \n    * **신용 등급:** 한국의 신용 등급은 안정적인 수준을 유지하고 있습니다.\n\n* **부정적인 측면:**\n    * **내수 부진:** 건설 투자가 부진하면서 내수 회복이 제약되고 있습니다. \n    * **고용:

In [37]:
# Manual input for chat history if you want to add something usefule to chat history.
chat_history.add_user_message("what's your role?")
chat_history.add_ai_message("I'm a helpful assistant")